In [1]:
import openai
import pandas as pd

In [2]:
#your key

In [3]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-5-nano",
    input="Write a one-sentence bedtime story about a unicorn."
)

print(response.output_text)


Under a velvet night sky, a gentle unicorn trotted to a quiet hill, where the stars hummed lullabies and the whole world drifted into a peaceful sleep.


In [4]:
from openai import OpenAI
client = OpenAI()

BASE_PROMPT = """You are a virtual assistant that helps users retrieve building information
from a BIM database and answer general questions about BIM technology and construction.

The BIM database consists of building objects in the following categories:
space, floor, door, window, stair, column, beam, furniture.

Your task is to classify the intent of the given query into exactly ONE of these categories.

Rules:
- Output ONLY the category name.
- Do NOT explain your reasoning.
- If multiple categories appear, choose the primary target.

Examples:
Q: what is the id of space Raum3?
A: space

Q: which floor has most number of windows?
A: floor

Q: where is window 2227 located in?
A: window

Q: How many stairs in this building are on the KG floor?
A: stair

Q: Tell me the GrossVolume number of the column whose OuterSurfaceArea is 1.8.
A: column

Q: Tell me the Length of beam named as Unterzug-1.
A: beam

Q: How many furnitures are with Space more than 0.20?
A: furniture
"""

def classify_intent(query: str) -> str:
    prompt = f"""{BASE_PROMPT}

Q: {query}
A:
"""

    response = client.responses.create(
        model="gpt-5",   
        input=prompt
    )

    return response.output_text.strip().lower().split()[0]


In [5]:
print(classify_intent("How many stairs are there on KG?"))

stair


## Intent recognition_GPT

In [6]:
df = pd.read_csv("val_data_all.csv") 
df.head()


,id,annotator,gpt测试结果,答案,Unnamed: 4,position,question,table_file,answer_coordinates,answer_text,aggregation_label,float_answer,Unnamed: 9,澶囨敞aggregation_label涓細\n0锛歂ONE\n1锛歋UM\n2锛欰VERAGE\n3: COUNT,label
0,NaN,NaN,1.0,NaN,NaN,NaN,What is the elevation of Ground in meters abov...,52Theparade(rvt2017)_floor.csv,"['(1, 2)']",['-46.5370539649164'],0,-46.537054,NaN,NaN,floor
1,NaN,NaN,0.0,167.000,产生幻觉,NaN,"How many windows are in the building, do you k...",161210Med_Dent_Clinic_Combined_floor.csv,"['(8, 7)', '(9, 7)', '(13, 7)']",['94.0'],1,94.000000,NaN,NaN,floor
2,NaN,NaN,1.0,NaN,NaN,NaN,What is the number of stairs on EG?,20170601_Mauer_BmB_floor.csv,"['(2, 4)']",['1.0'],0,1.000000,NaN,NaN,floor
3,NaN,NaN,1.0,NaN,NaN,NaN,How many stairs are there in 4th Floor?,OfficeBuilding_floor.csv,"['(2, 5)']",['5.0'],0,5.000000,NaN,NaN,floor
4,NaN,NaN,0.0,2.965,NaN,NaN,How high above the ground is Level 7 in relati...,Learningzonecorentin_floor.csv,"['(8, 2)']",['6.823'],0,6.823000,NaN,NaN,floor


In [7]:
predictions = []

for i, q in enumerate(df["question"]):
    try:
        pred = classify_intent(q)
        print(f"{i}: {pred}")   # 
    except Exception as e:
        print(f"Error at row {i}: {e}")
        pred = ""

    predictions.append(pred)


0: floor
1: window
2: stair
3: stair
4: floor
5: door
6: beam
7: floor
8: floor
9: floor
10: floor
11: window
12: beam
13: floor
14: column
15: stair
16: door
17: window
18: floor
19: floor
20: floor
21: floor
22: door
23: column
24: window
25: floor
26: door
27: floor
28: floor
29: stair
30: floor
31: door
32: space
33: window
34: floor
35: door
36: window
37: space
38: stair
39: column
40: floor
41: floor
42: floor
43: column
44: door
45: beam
46: door
47: floor
48: window
49: floor
50: space
51: space
52: door
53: stair
54: stair
55: window
56: floor
57: floor
58: column
59: floor
60: column
61: floor
62: floor
63: floor
64: door
65: door
66: door
67: floor
68: stair
69: space
70: beam
71: window
72: floor
73: window
74: furniture
75: floor
76: door
77: column
78: door
79: stair
80: door
81: window
82: floor
83: stair
84: column
85: furniture
86: stair
87: column
88: door
89: beam
90: space
91: door
92: door
93: window
94: floor
95: window
96: door
97: space
98: floor
99: door
100: 

In [10]:
df["gpt5_IR"] = predictions
df.to_csv("val_data_all_with_gpt5_IR.csv", index=False)


In [11]:
accuracy = (df["label"] == df["gpt5_IR"]).mean()
accuracy


0.49712643678160917

## TableQA

In [13]:
from openai import OpenAI
import pandas as pd
import time
import os

# ===============================
# 0. OpenAI Client
# ===============================
client = OpenAI()

# ===============================
TABLE_BASE_DIR = r"H:\XXX"

# ===============================
# 1. Table QA Prompt（System + Few-shot）
# ===============================
TABLE_QA_BASE_PROMPT = """ Please act as a BIM virtual assistant that retrieves information from a tabular BIM database.
Learn from the provided examples and answer user queries accordingly.

Rules:
- Use only the provided table.
- If the answer is a number, output ONLY the number.
- If the answer is text, output ONLY the text.
- If the answer cannot be found, output "not found".

Examples:

Tabular BIM database:

	Floor	Elevation	IfcDoor	IfcWindow	IfcSpace
0	Mur de fondation T.O	-300	1		
1	Ground	-46.53705396	10	13	
2	Niveau 1	0		2	8
3	Niveau 10	848.662946	6		
4	Niveau 12	2248.662946	2	9	
5	Niveau 9	2848.662946	2	14	
6	Niveau 13	4043.862946			3
7	Niveau 11	5543.862946			


User:
Q: What is the elevation of Ground in meters above sea level?
A: -46.53705396


Tabular BIM database:

	Floor	Elevation	IfcColumn	IfcDoor	IfcStair	IfcRailing	IfcWindow	IfcSpace	IfcStairFlight
0	MEP 1st Flr	0							
1	MEP 2nd Flr	4570							
2	MEP 1st Flr Supply	3545							
3	MEP 1st Flr Return	3100							
4	MEP 1st Flr Ceiling	2700							
5	MEP 2nd Flr Supply	8115							
6	MEP 2nd Flr Ceiling	7270							
7	MEP 2nd Flr Return	7600							
8	First Floor	0	71	304	4	13	32	158	7
9	Second Floor	4570	54	184		2	31	104	
10	Roof	8535							
11	Penthouse_High	9245							
12	TOF Footing	-1980							
13	Parapet _1	5334					31		
14	Center Radius	3320							
15	Roof Level	13500							
16	Level 7	7535							


User:
Q: How many windows are in the building, do you know?
A: 94.0

"""

# ===============================
def csv_to_table_text(csv_path: str) -> str:
    df = pd.read_csv(csv_path)
    return df.to_string(index=False)

# ===============================

# ===============================
def table_qa_from_csv(csv_path: str, question: str) -> str:
    table_text = csv_to_table_text(csv_path)

    prompt = f"""{TABLE_QA_BASE_PROMPT}

Tabular BIM database:
{table_text}

Q: {question}
A:
"""

    response = client.responses.create(
        model="gpt-5",
        input=prompt
    )

    return response.output_text.strip()


# ===============================
MAIN_CSV = "val_data_all.csv"
OUTPUT_CSV = "table_qa_with_gpt5_TQA.csv"

df = pd.read_csv(MAIN_CSV)


df["gpt5_TQA"] = ""

total = len(df)

for i, row in df.iterrows():
    question = row["question"]
    table_file = os.path.join(TABLE_BASE_DIR, row["table_file"])


    if df.at[i, "gpt5_TQA"] != "":
        continue

    if not os.path.exists(table_file):
        print(f"[{i+1}/{total}] Table not found: {table_file}")
        df.at[i, "gpt5_TQA"] = "table not found"
        continue

    try:
        answer = table_qa_from_csv(table_file, question)
        print(f"[{i+1}/{total}] {answer}")
    except Exception as e:
        print(f"[{i+1}/{total}]  Error: {e}")
        answer = ""

    df.at[i, "gpt5_TQA"] = answer

    time.sleep(0.2)

# ===============================
# 
# ===============================
df.to_csv(OUTPUT_CSV, index=False)

print("\n All done! Results saved to:", OUTPUT_CSV)


[1/348] ✅ -46.537054
[2/348] ✅ 94.0
[3/348] ✅ 1.0
[4/348] ✅ 5.0
[5/348] ✅ 6.823
[6/348] ✅ 2
[7/348] ✅ 5334.0
[8/348] ✅ 8
[9/348] ✅ 156.0
[10/348] ✅ 6.0
[11/348] ✅ 24500
[12/348] ✅ 22.0
[13/348] ✅ 260.0
[14/348] ✅ 8535.0
[15/348] ✅ 4
[16/348] ✅ 1.0
[17/348] ✅ Level 1
[18/348] ✅ 66.0
[19/348] ✅ 0.0
[20/348] ✅ 8
[21/348] ✅ 0.0
[22/348] ✅ -800.0
[23/348] ✅ 304.0
[24/348] ✅ 2.0
[25/348] ✅ 4.0
[26/348] ✅ 0.0
[27/348] ✅ 6.0
[28/348] ✅ 6.990284e-12
[29/348] ✅ 4
[30/348] ✅ Stair:Monolithic Stair:307242
[31/348] ✅ 0.0
[32/348] ✅ 2
[33/348] ✅ 2
[34/348] ✅ 14.0
[35/348] ✅ 2248.662946
[36/348] ✅ 74
[37/348] ✅ 5.0
[38/348] ✅ 198
[39/348] ✅ 15.0
[40/348] ✅ 1.0
[41/348] ✅ 0.0
[42/348] ✅ 3800.0
[43/348] ✅ 14000.0
[44/348] ✅ 13.0
[45/348] ✅ 546.0
[46/348] ✅ 18
[47/348] ✅ 8.0
[48/348] ✅ 4090.4
[49/348] ✅ 3700.0
[50/348] ✅ 30800.0
[51/348] ✅ 17.0
[52/348] ✅ 7600.0
[53/348] ✅ 4
[54/348] ✅ 10.0
[55/348] ✅ 1.0
[56/348] ✅ 283.0
[57/348] ✅ 4
[58/348] ✅ 0.0
[59/348] ✅ 80.0
[60/348] ✅ 0.0
[61/348] ✅ 124.0
[62/34